In [4]:
import tkinter
from tkinter import *
from tensorflow.keras.models import load_model
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import json
import random
import numpy as np

intents = json.loads(open('intents.json').read())
model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def bow(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i] = 1
                
    return (np.array(bag))

def predict_class(sentence):
    sentence_bag = bow(sentence)
    res = model.predict(np.array([sentence_bag]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #sort by probability
    results.sort(key=lambda x: x[1],reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
    return return_list
    

def get_response(ints):
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents:
        if(i['tag']==tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg)
    res =get_response(ints)
    return res
    
def send():
    msg = textentrybox.get("1.0",'end-1c').strip()
    textentrybox.delete('1.0','end')
    
    if msg!='':
        chathistory.config(state=NORMAL)
        chathistory.insert('end',"\nYou: "+msg+ "\n\n")
        
        res= chatbot_response(msg)
        chathistory.insert('end',"\nBot: "+res)
        chathistory.config(state=DISABLED)
        chathistory.yview('end')

base = Tk()
base.title("My_Bot")
base.geometry("400x500")
base.resizable(width=False,height=False)
#base.mainloop()

#chathistory textview

chathistory = Text(base, bd=0, bg='#CC99FF', font='Arial')
chathistory.config(state=DISABLED)

sendbutton = Button(base, font=('Arial',12,'bold'),text="Send",bg='#FF8D85',activebackground='#66cc66',fg='#4b3316',command=send)

textentrybox = Text(base,bd=0,bg='yellow',font='Arial')

chathistory.place(x=6,y=6,height=386,width=386)
textentrybox.place(x=128,y=400,height=80,width=265)
sendbutton.place(x=6,y=400,height=80,width=125)
base.mainloop()